# Methods of accessing data and metadata

With the ever-increasing volume of climate data, it is crucial to be able to easily and quickly search for and access datasets of interest. Below are a few tools that may be useful.

## STAC (SpatioTemporal Asset Catalog)

> "Enabling online search and discovery of geospatial assets"

[STAC](https://stacspec.org) is a specification aimed at standardizing the language around geospatial datasets in order to increase accessability and interoperability across many datasets. It is for use with data stored in the cloud. 

### Making data easily searchable
At its core, `STAC` provides a [JSON](https://www.json.org/json-en.html) file wrapper around any geospatial data (i.e. any data relating to the Earth). The goal of this wrapper file is to contain all relevant information that a user may want to search for when finding a dataset. In this way, `STAC` seeks to make all earth-related cloud-optimized datasets easily searchable, but does not provide the search tools themselves. `STAC` integrates well with a tool like `intake` that can search for and load the desired datasets.

(intake)=
## Intake

> "Taking the pain out of data access and distribution"

[Intake](https://intake.readthedocs.io/en/latest/) is a python package used to find, investigate, load, and disseminate data. 


### Data loading
`Intake` can be used to load many different types of data formats (e.g. tabular data, multi-dimensional data, etc.) into a python notebook or script using familiar containers (e.g. Pandas dataframes, Xarray DataArrays, etc.). Intake can work on local, remote, or cloud computing infrastructures, and is relatively fast due to its ability to integrate distirbuted computing (e.g. Dask).

There are a number of [plugins](https://intake.readthedocs.io/en/latest/plugin-directory.html#plugin-directory) that currently exist for different types of data. Several that may be of particular interest to the climate science community are:

- [intake-esm](https://intake-esm.readthedocs.io/en/latest/): for ESM catalog datasets (including CMIP and CESM datasets)
- [intake-geopandas](https://github.com/intake/intake_geopandas): for reading in geospatial datasets into a [geopandas](https://geopandas.org) dataframe
- [intake-stac](https://intake-stac.readthedocs.io/en/latest/): for SpatioTemporal Asset Catalogs ([STAC](https://stacspec.org)); the [Pangeo Data Catalog](https://pangeo.io/catalog.html) uses STAC to catalog its cloud-based datasets
- [intake-thredds](https://intake-thredds.readthedocs.io/en/latest/): to retrieve data from THREDDS data servers (used by NCAR for example)
- [intake-xarray](https://intake-xarray.readthedocs.io/en/latest/): to load datasets into [Xarray](http://xarray.pydata.org/en/stable/) containers

### NCI Data Collection Intake Catalogue

To access datasets stored at NCI ([National Computational Infrastructure](https://nci.org.au)), there is a dedicated intake cataloge: `nci-intake-catalogue`. Usage and other information can be found on the [nci-intake github site](https://github.com/coecms/nci-intake-catalogue).
Here we are showing how to use it to query and access CMIP6 data as an example.

```python
# Load intake module and the `nci` catalogue
import intake
cat = intake.cat.nci
list(cat)

   ['era5', 'era5_land', 'ecmwf', 'esgf', 'cosima', 'erai']

# CMIP6 is included in `esgf` which is itself a catalogue
# so we are using list() again to see the sub-catalogues
esgf = cat['esgf']
list(esgf)

   ['cmip5', 'cmip5_all', 'cmip5_gr1p5', 'cmip6', 'cmip6_all',
    'cmip6_gr1p5', 'cordex', 'cordex_all']

# NB you can also use esgf._entries to see a much more detailed description of `esgf` entries
esgf._entries
 
# NB cmip6_gr1p5 is cmip6 post-processed data and cmip6_all includes this and the original data
# We want `cmip6` original data only and we can look at its records as a pandas dataframe
cmip6 = cat['esgf'].cmip6
cmip6.df.head()

# `df.columns` lists all the columns that can be used to query the data
cmip6.df.columns

   Index(['project', 'activity_id', 'institution_id', 'source_id',
          ...

# Other useful attributes of a catalogue:
# - `description` 
# - `aggregation_info` how the data can be aggregated
cmip6.description
cmip6.aggregation_info

# The method unique() lists all the unique values for each column as a dictionary
cmip6.unique()

# Let's select a subset passing the search() method some constraints
subset = cmip6.search(activity_id='CMIP', experiment_id='historical', source_id='ACCESS-CM2', table_id='day', variable_id='tas')
subset
          
     cmip6 catalog with 5 dataset(s) from 20 asset(s):

     unique
     project	1
     activity_id	1
     ...
     path	20
     version	4

# to_dataset_dict() returns a dictionary listing all the datasets in our subset
dset_dict = subset.to_dataset_dict()
dset_dict

  --> The keys in the returned dictionary of datasets are constructed as follows:
	'project.activity_id.institution_id.source_id.experiment_id.member_id.table_id.variable_id.grid_label.version'
  100.00% [5/5 01:30<00:00]
  [9]:
  {'CMIP6.CMIP.CSIRO-ARCCSS.ACCESS-CM2.historical.r5i1p1f1.day.tas.gn.v20210607': <xarray.Dataset>
   Dimensions:    (time: 60265, bnds: 2, lat: 144, lon: 192)
   Coordinates:
    * time       (time) datetime64[ns] 1850-01-01T12:00:00 ... 2014-12-31T12:00:00
   ....
 
# Finally we can simply load a dataset using its key
ds = dset_dict['CMIP6.CMIP.CSIRO-ARCCSS.ACCESS-CM2.historical.r5i1p1f1.day.tas.gn.v20210607']
ds

   xarray.Dataset
   Dimensions:   time: 60265bnds: 2lat: 144lon: 192
   ...
```

### Other features

`Intake` (and many of the plugins) can also be used for a few other tasks:
- cataloging system for listing data sources, metadata, and parameters
- convenience functions that can be used to, among other things, distribute data catalogs 
- investigate data sources and create plots using a GUI

### Working with authorised catalogues
There is no direct way for `intake` to open and load authorised catalogues. Including the username and password in the URL helps open the catalogue, but further data processing using Dask generates an error message relating to the "nonnumeric port" in the URL. Nikhil Garg (Data61, CSIRO) advises on creating a [`.netrc` file](https://linux.die.net/man/5/netrc) and a [`.dodsrc` file](www.po.gso.uri.edu/tracking/tracking/vodhub/docs/dodsrc.html) in the home directory to resolve this issue. These files are used by the netCDF4 library; when intake uses Xarray and Dask to access the files, netCDF4 is utilised in the background. The `.netrc` file contains details about the machine, username, and password. It is a single-line file that can be written in the following format: `machine machine_name login user_ID password user_password`. The `.dodsrc` file points to the location of the `.netrc` file, and can be written as a single-line: `HTTP.NETRC=YourHomeDirectory/.netrc`. Note, when working on [NCI OOD](https://opus.nci.org.au/display/OOD), these files need to be created in the OOD home directory. Hint: According to Nikhil, this method works for any tool (e.g., R, cdo, nco). 

### Intake and distributed client
`Intake` is not working currently with a distributed client. You can avoid using distributed client or open the data catalogue link with Xarray to work around this issue, e.g., 
```python
ds = xarray.open_dataset("catalogue_link", chunks={'lat': 100, 'lon': 100, 'time': 500})
```

<div class="alert alert-info" role="alert">
<b>For dataset users:</b> if an intake catalog is already set up on your system, then all you need to know if the code to access the dataset you are interested in!
</div>

<div class="alert alert-warning" role="alert">
  <b>For dataset maintainers:</b> more specialized knowledge will be necessary.
</div>